In [ ]:
import os
from typing import TypedDict, List, Annotated
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_huggingface import HuggingFaceEndpoint



In [ ]:
class ChatState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]


In [21]:
def chat_node(state: ChatState) -> ChatState:
    llm = HuggingFaceEndpoint(
        repo_id="mistralai/Mistral-7B-Instruct-v0.2",
        task="text-generation",
        huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN")
    )

    user_input = input("User: ")
    if user_input.lower() in ["exit", "quit"]:
        return state

    state["messages"].append(HumanMessage(content=user_input))

    response = llm.invoke(state["messages"])
    if isinstance(response, str):
        bot_reply = response
    else:
        bot_reply = response.content

    print("Bot:", bot_reply)

    state["messages"].append(AIMessage(content=bot_reply))

    return state


In [22]:
graph = StateGraph(ChatState)
graph.add_node("chat", chat_node)
graph.add_edge(START, "chat")
graph.add_edge("chat", "chat")
app = graph.compile()


ValueError: Invalid reducer signature. Expected (a, b) -> c. Got (iterable=(), /)